In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
data = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

df_train = data.sample(frac=0.8, random_state=0)
df_valid = data.drop(df_train.index, axis=0)

max_ = df_train.min(axis=0)
min_ = df_train.max(axis=0)
df_train = (df_train - min_) / (max_ - min_)
df_valid = (df_valid - min_) / (max_ - min_)

x_train = df_train.drop(["quality"], axis=1)
x_valid = df_valid.drop(["quality"], axis=1)
y_train = df_train["quality"]
y_valid = df_valid["quality"]

In [5]:
from IPython.display import display

display(x_train.head(4))
print(x_train.shape)

In [8]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks

EPOCHS = 500
BATCH_SIZE=156

model = keras.Sequential([
    layers.InputLayer(input_shape=[11]),
    layers.Dense(units=512, activation="relu"),
    layers.Dense(units=512, activation="relu"),
    layers.Dense(units=512, activation="relu"),
    layers.Dense(units=1, activation="linear")
])

early_stopping_callback = callbacks.EarlyStopping(
        min_delta=0.001, 
        patience=20,
        restore_best_weights=True
)

model.compile(
    loss="mae",
    optimizer="adam"
)

In [9]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_valid, y_valid),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping_callback])

In [13]:
history_df = pd.DataFrame(history.history)

print(history_df["val_loss"].min())
history_df.loc[:, ["loss", "val_loss"]].plot()

# So, since the last epoch was 40th, then the lowest value of validation loss was at epoch 20
# Then it trained for 20 epochs (as defined by patience) more to see if there would be an improvements of at least (0.001 as defined by min_delta)
# Then, since at no point in the next 20 epochs the validation loss improved by at least 0.001, it restored the previous best weights it found (in epoch 20)